## Maximum Likelihood Estimation (MLE)

* <b>우도(likelihood)</b>
    + 확률 밀도 함수를 랜덤변수의 값  x 의 함수가 아닌 파라미터  θ 의 함수로 보는 것
    + 확률 분포로부터 특정한 샘플 값  x 가 발생하였을 때, 이 샘플 값  x 가 나오게 하는 파라미터  θ 의 가능성
    + 확률 분포로부터 특정한 샘플 값  x 가 발생하였을 때, 샘플 값  x 와 변수  θ 에서의는 확률(밀도함수)

이항분포의 확률함수는 다음과 같다.<br>
$$p(K=k)={n \choose k}θ^k(1−θ)^{n−x}$$

## Optimization via Gradient Descent

$$θ\longleftarrow θ-\alpha \nabla_\theta L(x;\theta)$$

θ와 데이타가 주어졌을때 손실 함수를 θ에 대해 미분한 것에 learning rate를 곱해주고 그것을 θ에서 빼서 업데이트 해준다.

## Overfitting

<img src="img/lab7img1.PNG" width="50%"/>

## Regularization

* Early Stopping : validation loss가 더이상 낮아지지 않을 때
* Reducing Network Size
* Weight Decay
* Dropout
* Batch Normalization

## Basic Approach to Train DNN

1. Make a neural network architecture.
2. Train and check that model is over-fitted.<br>
    a. If it is not, increase the model size (deeper and wider).<br>
    b. If it is, add regularization, such as drop-out, batch-normalization.
3. Repeat from step-2.

# Imports

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [13]:
# For reproducibility
torch.manual_seed(1)

# Training and Test Dataset

In [14]:
# |x_train| = (m, 3)
# |y_train| = (m,)
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [15]:
# |x_train| = (m', 3)
# |y_train| = (m',)
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

# Model

In [16]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3) # 3개의 element가진 1 dim벡터 받아서 3개의 element가진 1 dim벡터로 리턴
    def forward(self, x):
        return self.linear(x) # |x| = (m, 3) ->  (m, 3)

In [17]:
model = SoftmaxClassifierModel()

In [18]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Training

In [19]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.cross_entropy(prediction, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [20]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)
    
    print('Accuracy: {}% Cost: {:.6f}'.format(
         correct_count/len(y_test)*100, cost.item()
    ))

# Run

In [21]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 1.199645
Epoch    2/20 Cost: 1.142985
Epoch    3/20 Cost: 1.117769
Epoch    4/20 Cost: 1.100901
Epoch    5/20 Cost: 1.089523
Epoch    6/20 Cost: 1.079872
Epoch    7/20 Cost: 1.071320
Epoch    8/20 Cost: 1.063325
Epoch    9/20 Cost: 1.055720
Epoch   10/20 Cost: 1.048378
Epoch   11/20 Cost: 1.041245
Epoch   12/20 Cost: 1.034285
Epoch   13/20 Cost: 1.027478
Epoch   14/20 Cost: 1.020813
Epoch   15/20 Cost: 1.014279
Epoch   16/20 Cost: 1.007872
Epoch   17/20 Cost: 1.001586
Epoch   18/20 Cost: 0.995419
Epoch   19/20 Cost: 0.989365


In [22]:
test(model, optimizer, x_test, y_test)

Accuracy: 0.0% Cost: 1.425844


test 데이터로 실행시켰을때 cost값이 높아진 것을 확인할 수 있다. θ를 선택했을 때 이미 overfitting이 되어있다는 의미이다.

# Learning Rate

learning rate가 너무 크면 diverge 하면서 cost가 점점 늘어난다.(overshooting)

In [23]:
model = SoftmaxClassifierModel()

In [26]:
optimizer = optim.SGD(model.parameters(), lr=1e5)

In [27]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.280268
Epoch    1/20 Cost: 976950.750000
Epoch    2/20 Cost: 1279135.125000
Epoch    3/20 Cost: 1198379.000000
Epoch    4/20 Cost: 1098825.750000
Epoch    5/20 Cost: 1968197.625000
Epoch    6/20 Cost: 284763.218750
Epoch    7/20 Cost: 1532260.000000
Epoch    8/20 Cost: 1651504.000000
Epoch    9/20 Cost: 521878.500000
Epoch   10/20 Cost: 1397263.250000
Epoch   11/20 Cost: 750986.250000
Epoch   12/20 Cost: 918691.500000
Epoch   13/20 Cost: 1487888.125000
Epoch   14/20 Cost: 1582260.125000
Epoch   15/20 Cost: 685818.062500
Epoch   16/20 Cost: 1140048.750000
Epoch   17/20 Cost: 940566.562500
Epoch   18/20 Cost: 931638.250000
Epoch   19/20 Cost: 1971322.625000


learning rate가 너무 작으면 cost가 거의 줄어들지 않는다.

In [28]:
model = SoftmaxClassifierModel()

In [29]:
optimizer = optim.SGD(model.parameters(), lr= 1e-10)

In [30]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 3.187324
Epoch    2/20 Cost: 3.187324
Epoch    3/20 Cost: 3.187324
Epoch    4/20 Cost: 3.187324
Epoch    5/20 Cost: 3.187324
Epoch    6/20 Cost: 3.187324
Epoch    7/20 Cost: 3.187324
Epoch    8/20 Cost: 3.187324
Epoch    9/20 Cost: 3.187324
Epoch   10/20 Cost: 3.187324
Epoch   11/20 Cost: 3.187324
Epoch   12/20 Cost: 3.187324
Epoch   13/20 Cost: 3.187324
Epoch   14/20 Cost: 3.187324
Epoch   15/20 Cost: 3.187324
Epoch   16/20 Cost: 3.187324
Epoch   17/20 Cost: 3.187324
Epoch   18/20 Cost: 3.187324
Epoch   19/20 Cost: 3.187324


적절한 숫자로 시작해 발산하면 작게, cost가 줄어들지 않으면 크게 조정하자.

In [32]:
model = SoftmaxClassifierModel()

In [33]:
optimizer = optim.SGD(model.parameters(), lr= 1e-1)

In [34]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.341574
Epoch    1/20 Cost: 1.198802
Epoch    2/20 Cost: 1.150877
Epoch    3/20 Cost: 1.131977
Epoch    4/20 Cost: 1.116242
Epoch    5/20 Cost: 1.102514
Epoch    6/20 Cost: 1.089676
Epoch    7/20 Cost: 1.077479
Epoch    8/20 Cost: 1.065775
Epoch    9/20 Cost: 1.054511
Epoch   10/20 Cost: 1.043655
Epoch   11/20 Cost: 1.033187
Epoch   12/20 Cost: 1.023091
Epoch   13/20 Cost: 1.013356
Epoch   14/20 Cost: 1.003968
Epoch   15/20 Cost: 0.994917
Epoch   16/20 Cost: 0.986189
Epoch   17/20 Cost: 0.977775
Epoch   18/20 Cost: 0.969661
Epoch   19/20 Cost: 0.961836


# Data Preprocessing

<img src= "img/lab7img2.PNG" width="40%"/>

전처리 과정이 없다면 neural network는 값이 더 큰 칼럼에 집중하게 될 것이다. 

In [50]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90], 
                             [96, 98, 100], 
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

$$x_j^\prime={x_j-\mu_j\over \sigma_j}$$

여기서 $\sigma$는 standard deviation, $\mu$는 평균값이다.

In [51]:
mu = x_train.mean(dim=0)

In [52]:
sigma = x_train.std(dim=0)

In [53]:
norm_x_train = (x_train - mu) / sigma

In [54]:
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


# Training with Preprocessed Data

In [55]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)

In [56]:
model = MultivariateLinearRegressionModel()

In [57]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [58]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [59]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 29602.087891
Epoch    1/20 Cost: 18798.474609
Epoch    2/20 Cost: 11988.446289
Epoch    3/20 Cost: 7660.372559
Epoch    4/20 Cost: 4899.234863
Epoch    5/20 Cost: 3134.669678
Epoch    6/20 Cost: 2006.090454
Epoch    7/20 Cost: 1284.011230
Epoch    8/20 Cost: 821.937134
Epoch    9/20 Cost: 526.222534
Epoch   10/20 Cost: 336.964935
Epoch   11/20 Cost: 215.836212
Epoch   12/20 Cost: 138.308807
Epoch   13/20 Cost: 88.686241
Epoch   14/20 Cost: 56.922920
Epoch   15/20 Cost: 36.589779
Epoch   16/20 Cost: 23.571924
Epoch   17/20 Cost: 15.236008
Epoch   18/20 Cost: 9.896884
Epoch   19/20 Cost: 6.475713
